In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_6058/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_6058/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_30_7,0.942374,0.906153,0.866905,0.929998,0.024349,0.055711,0.070821,0.028001,0.090162,0.156040,0.934001,0.157209,41.430566,90.820237,"Hidden Size=[4], regularizer=0.02, learning_ra..."
1,model_1_30_3,0.942795,0.905732,0.863553,0.935146,0.024171,0.055961,0.072605,0.025942,0.090056,0.155470,0.934483,0.156635,41.445206,90.834877,"Hidden Size=[4], regularizer=0.02, learning_ra..."
2,model_1_30_0,0.942738,0.905567,0.862966,0.935792,0.024195,0.056059,0.072917,0.025684,0.090143,0.155548,0.934418,0.156713,41.443215,90.832886,"Hidden Size=[4], regularizer=0.02, learning_ra..."
3,model_1_29_7,0.942789,0.905031,0.863380,0.898800,0.024174,0.056378,0.072696,0.023038,0.090148,0.155478,0.934476,0.156643,41.444992,90.834664,"Hidden Size=[4], regularizer=0.02, learning_ra..."
4,model_1_31_2,0.942583,0.904643,0.861600,0.945459,0.024260,0.056608,0.073644,0.020436,0.090303,0.155758,0.934240,0.156925,41.437813,90.827484,"Hidden Size=[4], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,model_4_0_2,0.166549,-0.096315,-0.681530,-0.208588,0.352158,0.650820,0.894757,0.565052,0.287870,0.593429,-2.018445,0.597874,196.087349,477.899003,"Hidden Size=[24], regularizer=0.02, learning_r..."
875,model_5_0_1,0.151405,-0.105135,-0.558152,0.251523,0.358557,0.656056,0.829106,0.387547,0.354039,0.598796,11.337434,0.603282,292.051335,713.316178,"Hidden Size=[36], regularizer=0.02, learning_r..."
876,model_3_0_2,-0.005120,-0.205324,-0.972832,-0.079198,0.424693,0.715532,1.049762,0.611189,0.353175,0.651685,-0.584542,0.656567,99.712775,242.071240,"Hidden Size=[12], regularizer=0.02, learning_r..."
877,model_2_0_0,-0.156559,-0.299370,-1.116692,-0.231807,0.488681,0.771362,1.126311,0.705845,0.397672,0.699057,-0.534445,0.704294,67.432090,163.306158,"Hidden Size=[8], regularizer=0.02, learning_ra..."
